In [1]:
%run src/py/triangle.py
%run src/py/util.py
%run src/py/GLM.py
%run src/py/rockycore.py
%run src/py/TriangleTimeSeriesSplit.py
%run src/py/ModelInteractions.py


In [2]:
# creat a new rocky object
r = rocky()

# inspect object
print(r)

rocky(id='rocky0', mod=(), f=(), plot=(), t=(), rockylog=None)


In [3]:
# rename rocky object
r.rename('andys cool rocky object')
print(r)

rocky(id='andys cool rocky object', mod=(), f=(), plot=(), t=(), rockylog=None)


In [30]:
# add triangle
r.rename('rocky1')
r.SampleTri('taylor_ashe')
print(r)

rocky(id='rocky1', mod="tweedie", f=(), plot=(), t="paid_loss", rockylog=None)


In [31]:
# access triangle object
r.paid_loss

development_period         1          2          3          4          5   
accident_period                                                            
2001-01-01          357848.0  1124788.0  1735330.0  2218270.0  2745596.0  \
2002-01-01          352118.0  1236139.0  2170033.0  3353322.0  3799067.0   
2003-01-01          290507.0  1292306.0  2218525.0  3235179.0  3985995.0   
2004-01-01          310608.0  1418858.0  2195047.0  3757447.0  4029929.0   
2005-01-01          443160.0  1136350.0  2128333.0  2897821.0  3402672.0   
2006-01-01          396132.0  1333217.0  2180715.0  2985752.0  3691712.0   
2007-01-01          440832.0  1288463.0  2419861.0  3483130.0        NaN   
2008-01-01          359480.0  1421128.0  2864498.0        NaN        NaN   
2009-01-01          376686.0  1363294.0        NaN        NaN        NaN   
2010-01-01          344014.0        NaN        NaN        NaN        NaN   

development_period          6          7          8          9         10  
accident_pe

In [32]:
# access dataframe representation of triangle object
r.paid_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# access linear design matrix:
r.paid_loss.X_base.head(10)

,is_observed,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,development_period_0003,development_period_0004,development_period_0005,development_period_0006,development_period_0007,development_period_0008,development_period_0009,development_period_0010
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [8]:
# quick ATA factor summary
r.paid_loss.ata_summary()

development_period,1,2,3,4,5,6,7,8,9
2001,3.143,1.543,1.278,1.238,1.209,1.044,1.04,1.063,1.018
2002,3.511,1.755,1.545,1.133,1.084,1.128,1.057,1.086,
2003,4.448,1.717,1.458,1.232,1.037,1.12,1.061,,
2004,4.568,1.547,1.712,1.073,1.087,1.047,,,
2005,2.564,1.873,1.362,1.174,1.138,,,,
2006,3.366,1.636,1.369,1.236,,,,,
2007,2.923,1.878,1.439,,,,,,
2008,3.953,2.016,,,,,,,
2009,3.619,,,,,,,,
Vol Wtd,,,,,,,,,


In [9]:
# Triangle data type actually has a robust chain-ladder implementation that can be accessed

# this isn't important for right now, but it is there and it works well

In [10]:
r.AddModel?

Signature:
r.AddModel(
    id: str = None,
    model_class: str = 'tweedie',
    tri: triangle.Triangle = None,
    cal=False,
    n_validation=5,
)
Docstring:
Add a model to the ROCKY object.

Parameters
----------
id : str, optional
    The name to assign to the model.
    Default is None, which will assign a default name.
model_class : str, optional
    The model class to use.
    Default is "tweedie".
tri : Triangle, optional
    The triangle object to use.
    Default is None, which will use the first triangle in the ROCKY object.
cal : bool, optional
    Whether to use calendar periods as variables.
    Default is False.
n_validation : int, optional
    The number of validation folds to use for hyperparameter tuning.
    Default is 5.

Notes
-----
The following model classes are available:
    - "tweedie" (alias "glm") (starting with v0.0.1)
File:      c:\users\andyw\onedrive\documents\github\rocky\rocky-app\src\py\rockycore.py
Type:      method

In [11]:
# now add a GLM model to the triangle:
r.AddModel('tweedie', 'tweedie', 'paid_loss')
r.tweedie

tweedieGLM()

In [12]:
# r.tweedie.TuneFitHyperparameters()
r.tweedie.SetHyperparameters(alpha=0.3, power=2.3)

In [13]:
# r.tweedie.cv?

In [14]:
r.tweedie

tweedieGLM(alpha=0.3, power=2.3)

In [15]:
r.paid_loss.get_X('train')

,accident_period,development_period,cal,is_observed,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,...,development_period_0002,development_period_0003,development_period_0004,development_period_0005,development_period_0006,development_period_0007,development_period_0008,development_period_0009,development_period_0010,intercept
0,2001,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2002,1,2,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2003,1,3,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2004,1,4,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2005,1,5,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2006,1,6,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
6,2007,1,7,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
7,2008,1,8,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
8,2009,1,9,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
9,2010,1,10,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [16]:
r.tweedie.GetX('train')

,is_observed,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,development_period_0003,development_period_0004,development_period_0005,development_period_0006,development_period_0007,development_period_0008,development_period_0009,development_period_0010,intercept
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
7,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1
8,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1
9,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1


In [17]:
r.tweedie.Fit()

In [18]:
r.tweedie.PredictTriangle().query('development_period==10').yhat.round(0)

90    613383.0
91    600288.0
92    602973.0
93    605718.0
94    608238.0
95    610642.0
96    612470.0
97    613286.0
98    613081.0
99    612774.0
Name: yhat, dtype: float64

In [19]:
# reverse the series
atu = r.paid_loss.atu().tolist()
atu.reverse()

(r.paid_loss.diag().reset_index(drop=True) * pd.Series(atu)).round(0)

0    3901463.0
1    5433719.0
2    5378826.0
3    5297906.0
4    4858200.0
5    5111171.0
6    5660771.0
7    6784799.0
8    5642266.0
9    4969825.0
dtype: float64

In [20]:
r.tweedie.plot.obs_pred(log=True, color='cal')
# r.tweedie.plot.SetXLabels(r.paid_loss.get_X_id('train'))
# r.tweedie.plot.SetXForecast(r.paid_loss.get_X_id('test'))

In [21]:
r.tweedie.plot.residual('cal', log=True)

In [22]:
import plotly.express as px

In [23]:
r.paid_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
r.paid_loss.tri.iloc[:, 0]

accident_period
2001-01-01    357848.0
2002-01-01    352118.0
2003-01-01    290507.0
2004-01-01    310608.0
2005-01-01    443160.0
2006-01-01    396132.0
2007-01-01    440832.0
2008-01-01    359480.0
2009-01-01    376686.0
2010-01-01    344014.0
Name: 1, dtype: float64

In [25]:
px.line(r.tweedie.GetParameters().query('var_gp=="accident"'),
        x='parameter', y='cumsum')

In [26]:
r.paid_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
%run src/py/ModelInteractions.py

In [28]:
mi = ModelInteractions(r.tweedie, 2)

In [29]:
mi._get_X()

,,,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,...,dev_0009 * cal_0010,dev_0010 * cal_0002,dev_0010 * cal_0003,dev_0010 * cal_0004,dev_0010 * cal_0005,dev_0010 * cal_0006,dev_0010 * cal_0007,dev_0010 * cal_0008,dev_0010 * cal_0009,dev_0010 * cal_0010
accident_period,development_period,cal,,,,,,,,,,,,,,,,,,,,,
2001,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2002,1,2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2003,1,3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2004,1,4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2005,1,5,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006,1,6,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007,1,7,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008,1,8,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009,1,9,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
